<a href="https://colab.research.google.com/github/MateusBrasileiroNato/data_science/blob/main/Real_Estate_Data_Analysis_(Pandas).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

dados = pd.read_json(
    path_or_buf = 'realestates.json',
    orient = 'columns'
)

dados

In [ ]:
bairros = pd.read_excel(
    io = 'bairros.xlsx',
    sheet_name = 'Preço médio por tipo',
    usecols = 'C:E',
    header = 2,
    index_col = [0,1],
    names = ['Bairros', 'Tipo', 'Valor m2 / Bairro']
)

bairros

In [ ]:
# 1. NORMALIZES JSON FILE

dados_normal = pd.json_normalize(data = dados.normal)
dados_normal

In [ ]:
# 2. NORMALIZES JSON FILE (SAME AS ABOVE)

dados_normal['listings'].iloc[0]
dados_normal

In [ ]:
# EASIER WAY TO DO ALL THE STEPS ABOVE ONLY ONCE

dados_normal_listings = pd.json_normalize(data = dados.normal, sep = '_', record_path = ['listings'])
dados_normal_listings

In [ ]:
dados_highlights_listings = pd.json_normalize(data = dados.highlights, sep = '_', record_path = ['listings'])
dados_highlights_listings

In [7]:
# TURNING STRING TO LIST
# REMOVING "[]" AT "[1:-1]" 
# REPLACING "'" TO AN EMPTY SPACE
# SPLITTING ITEMS EVERY TIME ", " APPEARS

columns = dados_normal_listings.filter(like = 'imovel_caracteristicas').columns

for column in columns:
    
    dados_normal_listings[column] = dados_normal_listings[column].str[1:-1].str.replace("'", "").str.split(", ")
    dados_highlights_listings[column] = dados_highlights_listings[column].str[1:-1].str.replace("'", "").str.split(", ")



In [ ]:
dados_normal_listings.loc[0, 'imovel_caracteristicas_propriedade'][0]

In [ ]:
# TESTING DATAFRAME SIZE 

dados_normal_listings.columns == dados_highlights_listings.columns

In [ ]:
# APPENDING A TABLE TO ANOTHER

dados_listings = dados_normal_listings.append(dados_highlights_listings, ignore_index = True)
dados_listings

In [ ]:
# ANOTHER METHOD TO DO THE SAME

dados_listings = pd.concat([dados_normal_listings, dados_highlights_listings], ignore_index = True)
dados_listings

In [ ]:
dados_listings.head(2)

In [ ]:
bairros

In [ ]:
bairros_amostra = dados_listings['imovel_endereco_bairro']
bairros_amostra

In [ ]:
bairros_todos = bairros.index.get_level_values('Bairros')
bairros_todos

In [ ]:
bairros_amostra.shape

In [ ]:
bairros_amostra.unique()

In [ ]:
# CREATES SERIES WITH UNIQUE ITEMS FROM INDEX

bairros_amostra = pd.Series(bairros_amostra.unique())
bairros_amostra

In [ ]:
bairros_todos = pd.Series(bairros_todos.unique())
bairros_todos

In [ ]:
# VERIFICATION IF TABLES COMBINATION IS TRUE (SAME DATA)

bairros_amostra.isin(bairros_todos)

# INDEX 11 RETURNS FALSE

In [ ]:
# CORRECTING INDEX 11 (REPLACING "JACAREPAGUÁ" TO "FREGUESIA")

bairros.loc['Jacarepaguá']

In [ ]:
bairros.loc['Freguesia']

In [23]:
dados_listings['imovel_endereco_bairro'].replace('Freguesia (Jacarepaguá)', 'Freguesia', inplace = True)

In [ ]:
bairros_amostra = pd.Series(dados_listings['imovel_endereco_bairro'].unique())
bairros_amostra

In [ ]:
# VERIFICATION AFTER CORRECTION

bairros_amostra.isin(bairros_todos)

In [ ]:
dados_listings[['imovel_endereco_bairro', 'imovel_tipos_propriedade']]

In [ ]:
# TABLES MERGE

dados_listings = pd.merge(
    left = dados_listings,
    right = bairros,
    left_on = ['imovel_endereco_bairro', 'imovel_tipos_propriedade'], 
    right_index = True
)

dados_listings

In [ ]:
# ADDING INFORMATION

dados_listings['anuncio_descricao']

In [28]:
# TURNS NUMBERS TO INT

configuracao = dados_listings['anuncio_descricao'].str.extractall('(\d+)')

In [ ]:
# RENAMES COLUMNS

configuracao = configuracao.unstack().rename(columns = {0:'quartos', 1: 'suites', 2: 'banheiros'})
configuracao

In [ ]:
# REMOVES A LEVEL FROM TABLE

configuracao.droplevel(level = 0, axis = 1)

In [ ]:
dados_listings = pd.merge(dados_listings, configuracao, left_index = True, right_index = True)
dados_listings

In [31]:
# DEFINING NEW CLASSIFICATIONS

tipo_uso = {
    'Apartamento': 'Residencial',
    'Casa': 'Residencial',
    'Cobertura': 'Residencial',
    'Consultório': 'Comercial',
    'Imóvel': 'Comercial',
    'Loja': 'Comercial',
    'Sala Comercial': 'Comercial'
}

In [ ]:
dados_listings.imovel_tipos_propriedade.map(tipo_uso)

In [33]:
imovel_tipos_uso = dados_listings.imovel_tipos_propriedade.map(tipo_uso)

In [ ]:
dados_listings.insert(loc = 0, column = 'imovel_tipos_uso', value = imovel_tipos_uso)
dados_listings

In [35]:
# NUMERICAL VALUES

valor_minimo = dados_listings['anuncio_valores_venda'].min()

valor_maximo = dados_listings['anuncio_valores_venda'].max()

In [36]:
rotulos = ['Popular', 'Padrão', 'Alto Padrão']

In [ ]:
pd.cut(x = dados_listings['anuncio_valores_venda'], bins = 3, labels = rotulos)

In [38]:
classes = [valor_minimo, 400000, 2000000, valor_maximo]

In [ ]:
# INCLUDES MININUM VALUE

dados_listings['classe_valor'] = pd.cut(x = dados_listings['anuncio_valores_venda'], bins = 3, labels = rotulos, include_lowest = True)
dados_listings

In [40]:
# VALUE / M2

dados_listings['valor_m2'] = dados_listings['anuncio_valores_venda'] / dados_listings['imovel_area']
valor_m2 = lambda data: data['anuncio_valores_venda'] / data['imovel_area'] if data['imovel_area'] != 0 else 0
dados_listings['valor_m2'] = dados_listings.apply(valor_m2, axis = 1)

In [ ]:
dados_listings[['anuncio_valores_venda', 'imovel_area', 'valor_m2']].iloc[16:19]

In [ ]:
dados_listings['Piscina'] = dados_listings['imovel_caracteristicas_condominio'].apply(lambda x: 'Piscina' in x)
pd.set_option('display.max_colwidth', None)
dados_listings[['imovel_caracteristicas_condominio', 'Piscina']].head()

In [ ]:
# STATISTICS GROUPING

dados_listings[['anuncio_valores_venda', 'anuncio_valores_condominio', 'anuncio_valores_iptu']].agg(['sum', 'mean', 'std'])

In [ ]:
agrupamento = dados_listings[['imovel_tipos_propriedade', 'valor_m2']].groupby(by = 'imovel_tipos_propriedade')
agrupamento

In [ ]:
agrupamento.mean().round()

In [ ]:
agrupamento.agg(func = ['min', 'mean', 'max', 'std'])

In [ ]:
agrupamento = dados_listings[['imovel_tipos_propriedade', 'classe_valor', 'valor_m2']].groupby(by = ['imovel_tipos_propriedade', 'classe_valor'])
agrupamento.mean().round()

In [ ]:
agrupamento.agg(func = ['min', 'mean', 'max', 'std'])

In [ ]:
dados_listings.value_counts(subset = 'imovel_tipos_propriedade', normalize = True)

In [ ]:
dados_listings.value_counts(subset = ['imovel_tipos_uso', 'classe_valor'], normalize = True).to_frame(name = '%') * 100

In [ ]:
# TURNS LINES TO COLUMNS

bairros.unstack(level = 0)

In [ ]:
# TURNS COLUMNS TO LINES

tabelas_estatisticas = agrupamento.agg(func = ['min', 'mean', 'max', 'std'])
tabelas_estatisticas = tabelas_estatisticas.stack(dropna = False)
tabelas_estatisticas.unstack(level = 1)

In [ ]:
agrupamento = dados_listings[['imovel_tipos_propriedade', 'classe_valor', 'valor_m2']].groupby(by = ['imovel_tipos_propriedade', 'classe_valor'])

tabelas_estatisticas = agrupamento.agg(func = ['min', 'mean', 'max', 'std']).round(2)
tabelas_estatisticas = tabelas_estatisticas.droplevel(level = 0, axis = 'columns')
tabelas_estatisticas.reset_index(inplace = True)
tabelas_estatisticas.pivot(
    index = 'imovel_tipos_propriedade',
    columns = 'classe_valor',
    values = 'mean'
    )

In [ ]:
dados_listings.pivot_table(
    index = 'imovel_tipos_propriedade',
    columns = 'classe_valor',
    values = 'valor_m2',
    fill_value = '-',
    margins = True,
    margins_name = 'Média Geral',
    aggfunc = 'mean'
    )

In [ ]:
dados_listings.pivot_table(
    index = ['imovel_tipos_propriedade', 'classe_valor'],
    columns = ['anuncio_tipos_listagem', 'Piscina'],
    values = 'valor_m2',
    fill_value = '-',
    margins = True,
    margins_name = 'Média Geral',
    dropna = True,
    aggfunc = 'mean'
    )

In [74]:
# TABULATIONS

caracteristicas_propriedade = dados_listings['imovel_caracteristicas_propriedade']

In [75]:
caracteristicas_propriedade_itens = caracteristicas_propriedade.explode()

In [ ]:
caracteristicas_propriedade_itens.value_counts()

In [ ]:
pd.merge(
    left = caracteristicas_propriedade_itens.value_counts(),
    right = caracteristicas_propriedade_itens.value_counts(normalize = True),
    left_index = True,
    right_index = True
)

In [79]:
caracteristicas_propriedade_itens.where(
    cond = caracteristicas_propriedade_itens != '',
    other = 'Sem características',
    inplace = True
)

In [ ]:
# TURNS EMPTY VALUES TO "SEM CARACTERÍSTICAS"

pd.merge(
    left = caracteristicas_propriedade_itens.value_counts(),
    right = caracteristicas_propriedade_itens.value_counts(normalize = True),
    left_index = True,
    right_index = True
)

In [83]:
# DATAFRAME STYLIZATION

tabela_frequencias = pd.merge(
    left = caracteristicas_propriedade_itens.value_counts(sort = False),
    right = caracteristicas_propriedade_itens.value_counts(normalize = True, sort = False),
    left_index = True,
    right_index = True
)

In [84]:
tabela_frequencias.query("index != 'Sem Características'", inplace = True)

In [ ]:
tabela_frequencias.rename(columns = {
    'imovel_caracteristicas_propriedade_x': 'Frequências',
    'imovel_caracteristicas_propriedade_y': 'Percentual'
}, inplace = True)

tabela_frequencias.rename_axis('Características', inplace = True)

tabela_frequencias

In [ ]:
tabela_frequencias.style.format({'Percentual': '{:.2%}'})

In [ ]:
tabela_frequencias.style.format({'Percentual': '{:.2%}'}).bar(subset = 'Percentual', vmin = 0.05, color = 'blue')

In [ ]:
tabela_frequencias.style.format({'Percentual': '{:.2%}'}).bar(subset = 'Percentual', vmin = 0.05, color = 'blue') \
.applymap(lambda x: f"color: {'red' if x >= 35 else 'white'}", subset = 'Frequências') \
.applymap(lambda x: f"font-weight: {'bold' if x >= 35 else 'normal'}", subset = 'Frequências')

In [115]:
bairros_zona_sul = ['Ipanema', 'Botafogo', 'Catete', 'Copacabana', 'Lagoa', 'Flamengo', 'Gávea', 'Glória', 'Humaitá', 'Jardim Botânico', 'Laranjeiras',
                    'Leme', 'Urca', 'Vidigal', 'Cosme Velho', 'São Conrado', 'Leblon', 'Rocinha']

In [ ]:
tabela_zona_sul = bairros.unstack().query("index in " + str(bairros_zona_sul)).droplevel(level = 0, axis = 1)
tabela_zona_sul.style.format('R$ {:,.2f}').highlight_max(color = 'green').highlight_min(color = '#C26161')

In [ ]:
tabela_zona_sul[['Apartamento']].style.format('R$ {:,.2f}').background_gradient(cmap = 'Reds')

In [ ]:
tabela_zona_sul.style.format('R$ {:,.2f}').background_gradient(subset = ['Apartamento'], cmap = 'Greens') \
.background_gradient(subset = ['Cobertura'], cmap = 'Blues')

In [121]:
# TO EXCEL

tabela_zona_sul.style.format('R$ {:,.2f}').background_gradient(subset = ['Apartamento'], cmap = 'Greens') \
.background_gradient(subset = ['Cobertura'], cmap = 'Blues') \
.to_excel('Tabela Zona Sul.xlsx', engine = 'openpyxl')